Для анализа и построения модели вам доступен тренировочный датасет с 300 000 записями о клиентах банка и значениями целевых переменных для каждого из них. Описание факторов из датасета доступно в файле feature_description.xlsx.

Целевая переменная представлена в трёх столбцах:

target1 - прекращение финансовой активности клиента;
target2 - закрытие РКО;
total_target = max(target1, target2) - отток клиента из банка.
Вам нужно построить модель, которая предсказывает значение total_target. С помощью построенной модели нужно сделать предсказание на тестовой выборке из 100 000 записей.


In [1]:
import pandas as pd

from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

## Загрузка данных

In [2]:
train_df = pd.read_parquet('train.parquet')
test_df = pd.read_parquet('test.parquet')

In [3]:
train_df.head()

,id,rko_start_months,max_end_fact_fin_deals,max_end_plan_non_fin_deals,max_start_fin_deals,max_start_non_fin_deals,min_end_fact_fin_deals,min_end_plan_non_fin_deals,min_start_fin_deals,min_start_non_fin_deals,...,cnt_days_cred_g_oper_3m,sum_deb_h_oper_3m,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m,target_1,target_2,total_target
0,0,-1.279132,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.352516,-0.081310,0.210189,1.140808,0.647477,0.810887,1.909568,0,0,0
1,1,-1.142591,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.188287,-0.650616,-0.316563,-1.136536,-0.127326,-0.147319,-0.517805,0,0,0
2,2,1.812270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.323487,0.632731,2.541558,3.017482,0.033394,0.019972,0.578428,0,0,0
3,3,-0.479407,NaN,0.356677,NaN,-0.332867,NaN,-0.704164,NaN,0.905748,...,-0.323487,-0.003549,-0.219016,0.228624,-0.428185,-0.220932,-0.596108,0,0,0
4,4,-1.503690,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.323487,0.182726,-0.287299,-0.528921,-0.563335,-0.021506,-0.361201,0,0,0


In [4]:
test_df.head()

,id,rko_start_months,max_end_fact_fin_deals,max_end_plan_non_fin_deals,max_start_fin_deals,max_start_non_fin_deals,min_end_fact_fin_deals,min_end_plan_non_fin_deals,min_start_fin_deals,min_start_non_fin_deals,...,cnt_days_deb_g_oper_3m,sum_cred_g_oper_3m,cnt_cred_g_oper_3m,cnt_days_cred_g_oper_3m,sum_deb_h_oper_3m,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m
0,300000,-1.076009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.514610,-0.045603,-0.131653,-0.323487,-0.169940,-0.316563,-0.639934,-0.171600,-0.237763,-0.596108
1,300001,-0.335070,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.514610,-0.045603,-0.131653,-0.323487,-0.169940,-0.316563,-0.639934,-0.171580,-0.222361,-0.517805
2,300002,0.920834,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.463706,-0.045603,-0.131653,-0.323487,-0.169940,-0.316563,-0.639934,0.020943,-0.160754,-0.204596
3,300003,-0.772897,-0.740714,NaN,-0.635375,NaN,-0.595302,NaN,-0.738424,NaN,...,-0.514610,-0.045603,-0.131653,-0.323487,-0.137993,-0.277545,-0.381706,-0.171600,-0.237763,-0.596108
4,300004,-0.665342,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.209184,-0.045603,-0.131653,-0.323487,-0.139926,-0.277545,-0.381706,-0.158918,-0.222361,-0.517805


## Обработка данных

Для базовой модели отбросим отдельные таргеты и будем использовать только total_target.

In [5]:
train_df.drop(["id", "target_1", "target_2"], axis=1, inplace=True)

Преобразуем тип категориальных признаков

In [6]:
cat_cols = [
    'channel_code', 'city', 'city_type',
    'index_city_code', 'ogrn_month', 'ogrn_year',
    'branch_code', 'okved', 'segment'
]

In [7]:
train_df[cat_cols] = train_df[cat_cols].astype("category")

## Разбиение на train, validation

In [8]:
X = train_df.drop("total_target", axis=1)
y = train_df.total_target
x_train, x_val, y_train, y_val = train_test_split(X, y,
                                                  test_size=0.2,
                                                  random_state=42)

## Обучение базовой модели

В качестве базовой модели возьмем LGBMClassifier

In [9]:
model = LGBMClassifier(verbosity=-1, random_state=42)
model.fit(x_train, y_train)

LGBMClassifier(random_state=42, verbosity=-1)

In [10]:
y_pred = model.predict_proba(x_val)[:, 1]
roc_auc_score(y_val, y_pred)

0.8699164269179985

Качество получилось довольно неплохим, но его еще можно улучшить

## Выгрузка результатов

In [11]:
test_df[cat_cols] = test_df[cat_cols].astype("category")

In [12]:
test_score = model.predict_proba(test_df.drop("id", axis=1))[:, 1]

In [13]:
sample_submission_df = pd.read_csv("sample_submission.csv")

In [14]:
sample_submission_df.head()

,id,score
0,300000,0.5
1,300001,0.5
2,300002,0.5
3,300003,0.5
4,300004,0.5


In [15]:
sample_submission_df["score"] = test_score

In [16]:
sample_submission_df.head()

,id,score
0,300000,0.443094
1,300001,0.499971
2,300002,0.036863
3,300003,0.009825
4,300004,0.007027


In [17]:
sample_submission_df.to_csv("my_submission.csv", index=False)